In [ ]:
import open3d as o3d
import numpy as np

input_file = "centered.ply"
pcd = o3d.io.read_point_cloud(input_file)

vertices = np.asarray(pcd.points)
distances = np.linalg.norm(vertices, axis=1)
normalized_distances = (distances - np.min(distances)) / (np.max(distances) - np.min(distances))

colors = np.repeat(normalized_distances[:, np.newaxis], 3, axis=1)
colors = 1 - colors

pcd.colors = o3d.utility.Vector3dVector(colors)

pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(k=30)

distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

radius2 = o3d.utility.DoubleVector([radius, radius * 2])
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector(radius2))

o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


A fenti jó

In [ ]:
import open3d as o3d
import numpy as np
from numba import njit, prange
import cupy as cp

#@njit(parallel=True)
#def calculate_distances_numba(vertices):
#    distances = np.empty(vertices.shape[0])
#    for i in prange(vertices.shape[0]):
#        distances[i] = np.sqrt(vertices[i,0]**2 + vertices[i,1]**2 + vertices[i,2]**2)
#    return distances

def calculate_distances_cupy(vertices):
    vertices_gpu = cp.asarray(vertices)
    distances_gpu = cp.linalg.norm(vertices_gpu, axis=1)
    return cp.asnumpy(distances_gpu)

input_file = "centered.ply"
pcd = o3d.io.read_point_cloud(input_file)
vertices = np.asarray(pcd.points)


# distances = calculate_distances_numba(vertices)  # CPU paralelisation
distances = calculate_distances_cupy(vertices)  # GPU paralelisation

min_dist = np.min(distances)
max_dist = np.max(distances)
normalized_distances = (distances - min_dist) / (max_dist - min_dist)
colors = np.repeat(normalized_distances[:, np.newaxis], 3, axis=1)
colors = 1 - colors

pcd.colors = o3d.utility.Vector3dVector(colors)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(k=30)

o3d.visualization.draw_geometries([pcd])

random teszt: kb 20 perc

In [ ]:
import open3d as o3d
import numpy as np
import cupy as cp

input_file = "centered.ply"
pcd = o3d.io.read_point_cloud(input_file)

vertices = cp.asarray(pcd.points)
distances = cp.linalg.norm(vertices, axis=1).get()
normalized_distances = (distances - np.min(distances)) / (np.max(distances) - np.min(distances))

colors = np.repeat(normalized_distances[:, np.newaxis], 3, axis=1)
colors = 1 - colors
pcd.colors = o3d.utility.Vector3dVector(colors)

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
pcd.orient_normals_consistent_tangent_plane(k=30)

distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radii = o3d.utility.DoubleVector([radius, radius*2])

mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, radii)

vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(mesh)
ctr = vis.get_view_control()
ctr.set_front([0, 0, -1])
ctr.set_up([0, 1, 0])
mesh.compute_vertex_normals()
vis.update_renderer()
vis.run()
vis.destroy_window()